In [2]:
import os, config, pickle, nbimporter, random, _1_extract, numpy as np, pandas as pd
Species, Data = _1_extract.Species, _1_extract.Data

Importing Jupyter notebook from _1_extract.ipynb


# Processing the lxmx data

## Goals:
* construct projection matrix $P$ for data
* for each $P$ compute dominant & subdominant eigenvalues and their logs, $r, r_1$
* for each $P$ compute $T_c, T_g, V, r_{0a}, r_{0b}, d, G, K$ (see theory for reference)

In [3]:
SPECIES = config.load_pickle(os.path.join(config.OUTPUT_DIR, 'species.pkl'))

## Auxillary Classes

#### Processing
This class will handle all of the goals mentioned above and augment the Data object created in the data extraction step.

In [4]:
class Processing:
    
    @staticmethod
    def process(species):
        factor = 1
        if 'age in months' in species.notes: # for species with age reported in month, I divided the ages by 12
            factor = 12
        for data in species.data:
            matrix = data.matrix
            ages = data.ages / factor
            l_a = matrix['l(a)'].to_numpy()
            f_a = matrix['f(a)'].to_numpy()
            p_a = matrix['p(a)'].to_numpy()
            data.leslie = Processing.generate_leslie(p_a, f_a)
            eigs = Processing.compute_eig(data.leslie)
            data.eigen = {'vals': eigs[0], 'right': eigs[1], 'left': eigs[2], 'r_i': eigs[3]}
            data.eigen['damping'] = np.exp(data.eigen['r_i'][1] - data.eigen['r_i'][0])
            data.derivatives = Processing.compute_derivatives(l_a, f_a, ages, data.eigen)
            data.approximations = Processing.compute_approximations(data.derivatives, data.eigen)

            
    @staticmethod
    def generate_leslie(p_a, f_a):
        N = p_a.shape[0]
        leslie = np.zeros((N, N))
        leslie[0] = f_a
        np.fill_diagonal(leslie[1:, :-1], p_a)
        return leslie

    @staticmethod
    def compute_eig(leslie):
        #RIGHT
        r_vals, r_vecs = np.linalg.eig(leslie)
        ix_r = r_vals.argsort()
        ix_r = ix_r[::-1]
        r_vals = r_vals[ix_r]
        r_vecs = r_vecs[ix_r]
        #LEFT
        l_vals, l_vecs = np.linalg.eig(leslie.T)
        ix_l = l_vals.argsort()
        ix_l = ix_l[::-1]
        l_vals = l_vals[ix_l]
        l_vecs = l_vecs[ix_l]
        assert np.linalg.norm(l_vals - r_vals) < .00001 #check eigenvalues are the same
        r_eigs = r_vecs[:2]
        l_eigs = l_vecs[:2]
        vals = r_vals[:2]
        r_i = np.log(np.real(vals))
        return vals, r_eigs, l_eigs, r_i 
    
    def compute_approximations(deriv, eig):
        approx = {}
        approx['d'] = np.exp(-2*np.pi*deriv['V']/deriv['T_c']**3)
        approx['r_0a'] = r_0a = np.log(deriv['R_0'])/deriv['T_c'] #without dispersion
        approx['r_0b'] = r_0a + deriv['V']*np.log(deriv['R_0'])**2/(deriv['T_c']**3)  #with dispersion
        approx['r_1a'] = eig['r_i'][0] - 2 * np.pi / (deriv['T_c']**3)
        approx['s_1a'] =  2 * np.pi / deriv['T_c'] - np.pi * deriv['V'] * np.log(deriv['R_0'])/ (deriv['T_c']**3)
        return approx
    
    @staticmethod
    def compute_derivatives(l_a, f_a, ages, eigen):
        derivatives = {}
        derivatives['R_0'] = R_0 = np.sum(l_a * f_a)
        derivatives['T_c'] = T_c =  1 / R_0 * np.sum(ages * l_a * f_a)
        derivatives['V'] = V = 1 / R_0 * np.sum((ages - T_c)**2 * l_a * f_a)
        derivatives['G'] = G = 1 / (R_0 * V**1.5) * np.sum((ages - T_c)**3 * l_a * f_a)
        derivatives['K'] = K =  1 / (R_0 * V**2) * np.sum((ages - T_c)**4 * l_a * f_a)
        return derivatives
    
    @staticmethod
    def check_derivatives(species):
        for data in species.data:
            T_c_reported = data.reported['T_c'][1]
            R_0_reported = data.reported['R_0']
            V_reported = data.reported['V']
            exceptions = []
            if not (abs(data.derivatives['T_c'] - T_c_reported) < .01):
                exceptions.append('T_c calculated: {}, reported: {}'.format(data.derivatives['T_c'], T_c_reported))
            if not (R_0_reported is None or abs(data.derivatives['R_0'] - R_0_reported) < .01): 
                exceptions.append('R_0 calculated: {}, reported: {}'.format(data.derivatives['R_0'], R_0_reported))
            if not (V_reported is None or abs(data.derivatives['V'] - V_reported) < .01): 
                exceptions.append('V calculated: {}, reported: {}'.format(data.derivatives['V'], V_reported))
            if len(exceptions) > 0:
                combined = '; '
                combined = combined.join(exceptions)
                raise Exception('For {}, {}'.format(species.name, combined))
                

In [5]:
for s in SPECIES:
    Processing.process(SPECIES[s])
    try:
        Processing.check_derivatives(SPECIES[s])
    except Exception as e:
        print('**********')
        print(e)

**********
For Spermophilus lateralis, T_c calculated: 3.3807044887780555, reported: 3.511; R_0 calculated: 0.9623999999999999, reported: 0.998; V calculated: 2.2432498777603995, reported: 2.618
**********
For Lepus europaeus, T_c calculated: 1.1066552581379423, reported: 1.217; V calculated: 0.09527991404947117, reported: 0.264
**********
For Capreolus capreolus, T_c calculated: 4.555876774257954, reported: 4.754; V calculated: 3.9787695061941166, reported: 4.814
**********
For Pteropus conspicillatus, T_c calculated: 4.001943479485495, reported: 4.76; V calculated: 4.921872103086531, reported: 3.302
**********
For Lynx rufus, T_c calculated: 2.169733372399648, reported: 2.67
**********
For Arctocephalus australis, T_c calculated: 6.323251800587961, reported: 5.823
**********
For Zapus princeps, T_c calculated: 1.9642274633018801, reported: 2.279; V calculated: 0.6142581144251301, reported: 1.1


/home/kaan/anaconda3/envs/bio172_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/kaan/anaconda3/envs/bio172_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars


In [9]:
s = 'Zapus princeps'

In [6]:
s = 'Spermophilus townsendii'

In [10]:
SPECIES[s]

Zapus princeps: 1 data entries, notes: []

In [11]:
pd.DataFrame(SPECIES[s].data[0].matrix)

,l(a),f(a),p(a)
age,,,
0.0,1.000,0.00,0.307000
1.0,0.307,0.85,0.951140
2.0,0.292,1.91,0.431507
3.0,0.126,1.20,0.119048
4.0,0.015,1.30,0.333333
5.0,0.005,1.00,1.000000
6.0,0.005,1.00,0.000000


In [12]:
SPECIES[s].data

[Falk & Millar 87]

In [14]:
SPECIES[s].data[0].leslie

array([[0.        , 0.85      , 1.91      , 1.2       , 1.3       ,
        1.        , 1.        ],
       [0.307     , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.95114007, 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.43150685, 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.11904762, 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.33333333,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        ]])

In [15]:
SPECIES[s].data[0].approximations

{'d': 0.6009289280988288,
 'r_0a': -0.00032083785873202974,
 'r_0b': -0.0003208056680378825,
 'r_1a': -0.8293080461333774,
 's_1a': 3.198967854115502}

In [16]:
SPECIES[s].data[0].eigen

{'vals': array([0.99978743+0.j        , 0.14763761+0.23169683j]),
 'right': array([[-0.91447366+0.j        ,  0.77179484+0.j        ,
          0.77179484-0.j        ,  0.05039642+0.2265691j ,
          0.05039642-0.2265691j ,  0.06721465-0.01805052j,
          0.06721465+0.01805052j],
        [-0.00457723+0.j        , -0.01233822+0.01884284j,
         -0.01233822-0.01884284j,  0.06489139+0.16356494j,
          0.06489139-0.16356494j,  0.11946809+0.18748866j,
          0.11946809-0.18748866j]]),
 'left': array([[-0.18590287+0.j        , -0.10784803-0.19173828j,
         -0.10784803+0.19173828j, -0.00893227+0.10288732j,
         -0.00893227-0.10288732j, -0.03361183+0.02223882j,
         -0.03361183-0.02223882j],
        [-0.37192433+0.j        ,  0.28268141+0.28254025j,
          0.28268141-0.28254025j, -0.55931906+0.00222515j,
         -0.55931906-0.00222515j,  0.4576896 +0.42582764j,
          0.4576896 -0.42582764j]]),
 'r_i': array([-2.12596590e-04, -1.91299459e+00]),
 'damping': 0.

In [127]:
def export_to_csv(species):
    """
    format of the exported csv file will be: 
    <identifiers:[species_name, dataset_tag], 
    eigen:[r_0, r_1, s_1, damping],
    derivatives: [R_0, T_c, V, G, >],
    approximations: [d, r_0a, r_0b, r_1a, s_1a]>
    
    right_vec and left_vecs will be comma-concanetaed
    """
    filename = 'processed.csv'
    table = []
    columns = ['species-name', 'dataset-tag', 
               'r_0', 'r_1', 's_1', 'damping',
               'R_0', 'T_c', 'V', 'G',
               'd', 'r_0a', 'r_0b', 'r_1a', 's_1a']
    for s in species:
        for data in species[s].data:
            identifiers = [s, str(data)]
            
            eigen = [data.eigen['r_i'][0], data.eigen['r_i'][1], np.imag(data.eigen['vals'][1]), 
                     data.eigen['damping']]
            
            derivatives = [data.derivatives['R_0'], data.derivatives['T_c'], 
                           data.derivatives['V'], data.derivatives['G']]
            
            approximations = [data.approximations['d'], data.approximations['r_0a'],
                             data.approximations['r_0b'], data.approximations['r_1a'],
                             data.approximations['s_1a']]
            
            table.append(identifiers + eigen + derivatives + approximations)
            
    df = pd.DataFrame(table)
    df.columns = columns
    return df

In [128]:
df = export_to_csv(SPECIES)

In [129]:
df.to_csv(os.path.join(config.OUTPUT_DIR, 'processed.csv'))

## Working with the Species objects

You can acccess the calculated derivatives as follows:

In [130]:
s = random.choice(list(SPECIES.keys()))

In [131]:
SPECIES[s]

Godley (stationary): 1 data entries, notes: []

In [132]:
SPECIES[s].data[0].eigen.keys()

dict_keys(['vals', 'right', 'left', 'r_i', 'damping'])

In [133]:
SPECIES[s].data[0].eigen['vals']

array([0.99089396+0.j        , 0.58665544+0.43938025j])

In [134]:
SPECIES[s].data[0].eigen['r_i']

array([-0.00914775, -0.53331761])

In [135]:
SPECIES[s].data[0].eigen['damping']

0.5920466412204611

In [136]:
SPECIES[s].data[0].derivatives

{'R_0': 0.927374,
 'T_c': 5.368981662198853,
 'V': 5.8839824913165,
 'G': 0.71153350199103,
 'K': 2.7762671258579386}

In [137]:
SPECIES[s].data[0].approximations

{'d': 0.7875112456898881,
 'r_0a': -0.01404332283649303,
 'r_0b': -0.013827190767604315,
 'r_1a': -0.04974570069224413,
 's_1a': 1.179280543115519}

In [138]:
SPECIES[s].data[0].reported

{'T_c': [None, 5.369], 'V': 5.884, 'R_0': None}

In [139]:
save = True
if save:
    config.save_pickle(SPECIES, os.path.join(config.OUTPUT_DIR, 'species_processed.pkl'))

## Bugs

In [140]:
Processing.process(SPECIES['Ursus arctos horribilis'])

/home/kaan/anaconda3/envs/bio172_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/kaan/anaconda3/envs/bio172_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars


In [142]:
df[df['r_0'] == - np.inf]

,species-name,dataset-tag,r_0,r_1,s_1,damping,R_0,T_c,V,G,d,r_0a,r_0b,r_1a,s_1a
49,Ursus arctos horribilis,Knight & Eberhardt 85,-inf,-inf,0.0,NaN,0.698387,10.317704,16.907862,0.533414,0.907810,-0.034793,-0.032809,-inf,0.626332
53,Cebus olivaceus large groups,Robinson & O'Brien 91,-inf,-inf,0.0,NaN,3.725395,17.817908,53.485321,0.358324,0.942322,0.073812,0.090166,-inf,0.313567
54,Cebus olivaceus small groups,Robinson & O'Brien 91,-inf,-inf,0.0,NaN,1.885453,16.051621,56.095486,0.564275,0.918309,0.039508,0.044963,-inf,0.364414
55,Equus asinus,Saltz 95,-inf,-inf,0.0,NaN,2.099600,7.609926,17.928334,0.293129,0.774444,0.097471,0.119854,-inf,0.730857
56,Spermophilus townsendii,Smith 85,-inf,-inf,0.0,NaN,1.534000,1.688396,0.715159,1.054114,0.393137,0.253423,0.280626,-inf,3.521659
